In [2]:
import tvm
import tvm.testing
from tvm import te
import numpy as np

# 全局环境定义

tgt_host = "llvm"
# 如果启用了GPU，则将其更改为相应的GPU，例如：cuda、opencl、rocm
tgt = "cuda"

In [3]:
n = te.var("n") # 定义符号变量 n
A = te.placeholder((n,), name="A")  # 定义占位符张量
B = te.placeholder((n,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")    # 计算规则
print(type(C))

<class 'tvm.te.tensor.Tensor'>


In [4]:
######################################################################
# 调度计算
# 虽然上面的几行描述了计算规则，但是我们可以用很多方法来计算C，因为C可以在轴上用数据并行的方式来计算。TVM要求用户提供一个称为schedule的计算描述。
# schedule是程序中变换计算循环的一组集合。在我们构造了schedule之后，默认情况下，schedule以串行方式按行的主要顺序计算C。
#
# .. code-block:: c
#
#   for (int i = 0; i < n; ++i) {
#     C[i] = A[i] + B[i];
#   }
#
s = te.create_schedule(C.op)

######################################################################
# 我们调用`te.create_schedule`来创建scheduler，然后使用split构造来拆分C的第一个轴，
# 这将把原来的一个迭代轴拆分成两个迭代轴的乘积
#
# .. code-block:: c
#
#   for (int bx = 0; bx < ceil(n / 64); ++bx) {
#     for (int tx = 0; tx < 64; ++tx) {
#       int i = bx * 64 + tx;
#       if (i < n) {
#         C[i] = A[i] + B[i];
#       }
#     }
#   }
#
bx, tx = s[C].split(C.op.axis[0], factor=64)

In [6]:
# 最后，我们将迭代轴bx和tx绑定到GPU计算grid中的线程。这些是特定于GPU的构造，允许我们生成在GPU上运行的代码。
#
if tgt == "cuda" or tgt == "rocm" or tgt.startswith("opencl"):
    s[C].bind(bx, te.thread_axis("blockIdx.x"))
    s[C].bind(tx, te.thread_axis("threadIdx.x"))

######################################################################
# Compilation
# 上面我们已经完成了指定scheduler，接下来我们就可以将上面的所有代码编译成一个TVM的函数了。
# 默认情况下，TVM会将其编译成一个类型擦除函数，可以直接从Python端调用。下面我们使用`tvm,build`来创建一个编译函数，
# 编译函数接收scheduler，函数签名（包含输入输出）以及我们需要编译到的目标语言。编译`fadd`的结果是一个GPU设备函数
# （如果涉及GPU）以及一个调用GPU函数的host端包装器。`fadd`是生成的主机包装函数，它在内部包含对生成的设备函数的引用。
#
Tar = tvm.target.Target(target=tgt, host=tgt_host)
fadd = tvm.build(s, [A, B, C], Tar, name="myadd")

[14:16:11] /home/parker/workspace/tvm/src/te/schedule/schedule_lang.cc:184: Warning: Axis iter_var(i.outer, ) is already bind to another thread iter_var(blockIdx.x, , blockIdx.x)
[14:16:11] /home/parker/workspace/tvm/src/te/schedule/schedule_lang.cc:184: Warning: Axis iter_var(i.inner, ) is already bind to another thread iter_var(threadIdx.x, , threadIdx.x)


In [12]:
######################################################################
# 编译后的TVM函数公开了一个简洁的C API，可以被任何语言调用。TVM在python中提供了一个最小
# 的array API来帮助快速测试和原型开发。阵列API基于DLPack标准。这个array API基
# 于https://github.com/dmlc/dlpack 标准。要运行这个函数，首先需要创建一个GPU context，
# 然后使用`tvm.nd.array`将数据拷贝到GPU，再使用我们编译好的函数`fadd`来执行计算，最后
# `asnumpy()`将GPU端的array拷贝回CPU使用numpy进行计算，最后比较两者的差距。这部分的代码如下：
#
device = tvm.device(tgt, 0)

n = 1024
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), device)
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), device)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), device)
print(a, a.dtype, a.device)

fadd(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())


[0.39465863 0.49697846 0.29477423 ... 0.15670976 0.33382186 0.32561234] float32 cuda(0)


In [13]:
######################################################################
# Inspect the Generated Code
# --------------------------
# You can inspect the generated code in TVM. The result of tvm.build
# is a TVM Module. fadd is the host module that contains the host wrapper,
# it also contains a device module for the CUDA (GPU) function.
#
# The following code fetches the device module and prints the content code.
#

if tgt == "cuda" or tgt == "rocm" or tgt.startswith("opencl"):
    dev_module = fadd.imported_modules[0]
    print("-----GPU code-----")
    print(dev_module.get_source())
else:
    print(fadd.get_source())

-----GPU code-----

#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(64) myadd_kernel0(float* __restrict__ C, float* __restrict__ A, float* __restrict__ B, int n, int stride, int stride1, int stride2) {
  if (((int)blockIdx.x) < (n >> 6)) {
    C[((((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride2))] = (A[((((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride))] + B[((((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride1))]);
  } else {
    if (((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) < n) {
      C[((((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride2))] = (A[((((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride))] + 

In [14]:
######################################################################
# Save Compiled Module
# --------------------
# Besides runtime compilation, we can save the compiled modules into
# a file and load them back later. This is called ahead of time compilation.
#
# The following code first performs the following steps:
#
# - It saves the compiled host module into an object file.
# - Then it saves the device module into a ptx file.
# - cc.create_shared calls a compiler (gcc) to create a shared library
#
from tvm.contrib import cc
from tvm.contrib import utils

temp = utils.tempdir()
fadd.save(temp.relpath("myadd.o"))
if tgt == "cuda":
    fadd.imported_modules[0].save(temp.relpath("myadd.ptx"))
if tgt == "rocm":
    fadd.imported_modules[0].save(temp.relpath("myadd.hsaco"))
if tgt.startswith("opencl"):
    fadd.imported_modules[0].save(temp.relpath("myadd.cl"))
cc.create_shared(temp.relpath("myadd.so"), [temp.relpath("myadd.o")])
print(temp.listdir())

['myadd.so', 'myadd.ptx', 'myadd.tvm_meta.json', 'myadd.o']


In [15]:
######################################################################
# Load Compiled Module
# --------------------
# We can load the compiled module from the file system and run the code.
# The following code loads the host and device module separately and
# re-links them together. We can verify that the newly loaded function works.
#
fadd1 = tvm.runtime.load_module(temp.relpath("myadd.so"))
if tgt == "cuda":
    fadd1_dev = tvm.runtime.load_module(temp.relpath("myadd.ptx"))
    fadd1.import_module(fadd1_dev)

if tgt == "rocm":
    fadd1_dev = tvm.runtime.load_module(temp.relpath("myadd.hsaco"))
    fadd1.import_module(fadd1_dev)

if tgt.startswith("opencl"):
    fadd1_dev = tvm.runtime.load_module(temp.relpath("myadd.cl"))
    fadd1.import_module(fadd1_dev)

fadd1(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())

In [16]:
######################################################################
# Pack Everything into One Library
# --------------------------------
# In the above example, we store the device and host code separately.
# TVM also supports export everything as one shared library.
# Under the hood, we pack the device modules into binary blobs and link
# them together with the host code.
# Currently we support packing of Metal, OpenCL and CUDA modules.
#
fadd.export_library(temp.relpath("myadd_pack.so"))
fadd2 = tvm.runtime.load_module(temp.relpath("myadd_pack.so"))
fadd2(a, b, c)
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())